In [ ]:
import subprocess
import neutralb1.utils

WORKSPACE_DIR = neutralb1.utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 

This notebook was run at commit:
``

# Acceptance-Correcting Projected Moments
When partial wave fits are performed to data that includes detector acceptance effects, we typically perform an "acceptance correction" to remove those effects so that we obtain the *true physics* of the decay. If we want to project out moments from partial wave results, these partial waves must be acceptance corrected, otherwise the moments will be measured quantities that aren't of particular interest to us. In this study we'll verify that we are projecting and acceptance correcting properly by comparing moment values obtained from fits to **thrown** Monte Carlo (no detector acceptance) vs **accepted** Monte Carlo, were we've applied detector acceptance and other cuts to the thrown data.

By performing partial wave fits in both datasets, we can check whether the acceptance-corrected projected moments of the accepted dataset match the projected moments of the thrown dataset. We also have access to the fitted moments in the thrown case, which can provide a further check that the projection is working well. Unfortunately, the fitted moments currently have no method to be
acceptance-corrected, and so we can not fit moments to the accepted dataset.

This notebook follows naturally from the previous [moment verification study](./verify_moment.ipynb)

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import scipy.stats

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/input-output-tests/proj-acc-correct"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

# TODO: load in matploblib style then adjust subsequent plot looks

Lets print out what parameters we've submitted the fit with. These are the same for thrown and accepted MC, except that `data_option` is changed to `'_mc'`. These fits are then run with `uv run submit submission.YAML`

In [ ]:
%%bash
cat $STUDY_DIR/submission.YAML

We then transfer only the necessary results of our fits to our study directory. First the thrown directory

In [ ]:
%%bash 
cd /lustre24/expphy/volatile/halld/home/kscheuer/ampToolsFits/omegapi/allPeriods/PARA_0/ver03.1_mcthrown/ver03/1m_1p_iso/recoil_pi_mass_0.0/t_0.10-0.20/mass_1.200-1.220/
cp best.csv "$STUDY_DIR/csv/thrown.csv"
cp data.csv "$STUDY_DIR/csv/thrown_data.csv"
cp best_projected_moments.csv "$STUDY_DIR/csv/thrown_projected_moments.csv"
cp distributions/angles.pdf "$STUDY_DIR/thrown_angles.pdf"

cp truth/best.csv "$STUDY_DIR/csv/thrown_truth.csv"
cp truth/best_projected_moments.csv "$STUDY_DIR/csv/thrown_truth_projected_moments.csv"

And now for the accepted MC

In [ ]:
%%bash
cd /lustre24/expphy/volatile/halld/home/kscheuer/ampToolsFits/omegapi/allPeriods/PARA_0/ver03.1_mc/ver03/1m_1p_iso/recoil_pi_mass_0.0/t_0.10-0.20/mass_1.200-1.220/
cp best.csv "$STUDY_DIR/csv/acceptance.csv"
cp best_corrected.csv "$STUDY_DIR/csv/acceptance_corrected.csv"
cp data.csv "$STUDY_DIR/csv/acceptance_data.csv"
cp rand/rand_acceptance_corrected.csv "$STUDY_DIR/csv/rand_acceptance_corrected.csv"
cp rand/rand_projected_moments.csv "$STUDY_DIR/csv/acceptance_rand_projected_moments.csv"
cp bootstrap/bootstrap.csv "$STUDY_DIR/csv/acceptance_bootstrap.csv"
cp bootstrap/bootstrap_corrected.csv "$STUDY_DIR/csv/acceptance_corrected_bootstrap.csv"
cp bootstrap/bootstrap_projected_moments.csv "$STUDY_DIR/csv/acceptance_bootstrap_projected_moments.csv"

cp best_projected_moments.csv "$STUDY_DIR/csv/acceptance_projected_moments.csv"
cp distributions/angles.pdf "$STUDY_DIR/acceptance_angles.pdf"


cp truth/best.csv "$STUDY_DIR/csv/acceptance_truth.csv"
cp truth/best_corrected.csv "$STUDY_DIR/csv/acceptance_corrected_truth.csv"
cp truth/best_projected_moments.csv "$STUDY_DIR/csv/acceptance_truth_projected_moments.csv"

Lets now load in our data files into pandas dataframes

In [ ]:
thrown_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown.csv")
thrown_data_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_data.csv")
thrown_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_projected_moments.csv")

truth_thrown_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_truth.csv")
truth_thrown_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_truth_projected_moments.csv")


acc_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance.csv")
acc_corrected_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_corrected.csv")
acc_data_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_data.csv")
acc_corrected_rand_df = pd.read_csv(f"{STUDY_DIR}/csv/rand_acceptance_corrected.csv")
acc_rand_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_rand_projected_moments.csv")
acc_bootstrap_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_bootstrap.csv")
acc_corrected_bootstrap_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_corrected_bootstrap.csv")
acc_bootstrap_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_bootstrap_projected_moments.csv")
acc_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_projected_moments.csv")

truth_acc_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_truth.csv")
truth_acc_corrected_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_corrected_truth.csv")
truth_acc_corrected_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_truth_projected_moments.csv")

It'll be much easier to manage the thrown and accepted signal MC results with the result manager

In [ ]:
thrown_result = ResultManager(
    fit_df = thrown_waves_df,
    data_df = thrown_data_df,
    proj_moments_df=thrown_moments_df,    
    truth_df = truth_thrown_waves_df,
    truth_proj_moments_df=truth_thrown_moments_df,
)

# projected moments are always acceptance corrected currently, so they're not included in this result
acc_result = ResultManager(
    fit_df = acc_waves_df,
    data_df = acc_data_df,
    bootstrap_df = acc_bootstrap_df,
    truth_df = truth_acc_waves_df,
)

acc_corrected_result = ResultManager(
    fit_df = acc_corrected_waves_df,
    data_df = acc_data_df,
    randomized_df = acc_corrected_rand_df,
    randomized_proj_moments_df = acc_rand_moments_df,
    bootstrap_df = acc_corrected_bootstrap_df,
    proj_moments_df = acc_moments_df,
    bootstrap_proj_moments_df = acc_bootstrap_moments_df,
    truth_df = truth_acc_corrected_waves_df,
    truth_proj_moments_df = truth_acc_corrected_moments_df,
)

thrown_result.preprocess()
acc_result.preprocess()
acc_corrected_result.preprocess()

## Analysis

### Fits to Distributions
Let's first make sure the angular distributions look alright

In [ ]:
utils.display_pdf(f"{STUDY_DIR}/thrown_angles.pdf", page=0, resolution=150)
utils.display_pdf(f"{STUDY_DIR}/acceptance_angles.pdf", page=0, resolution=150)

### Checking the Acceptance Correction via Truth Fits
We can see how the acceptance correction effects our results by comparing the truth fits in the thrown and accepted cases. A "truth" fit is one where the production coefficients are fixed to their generated values, and only a common scale factor is allowed to float so that the intensity can be adjusted to the number of events. This is because the generated production coefficients are sensitive to the intensity they were fit with, and so generated or accepted MC will have a different number of events. In either thrown or accepted cases, the production coefficients will be the same, just with a modified scale factor.

#### Partial Wave Fits
Let's first see how the waves were effected by comparing the ratio $A_{\text{acc-corrected}} / A_{\text{thrown}}$

In [ ]:
amplitudes = utils.get_coherent_sums(acc_corrected_result.truth_df)["eJPmL"]

ratios = []
for amp in amplitudes:
    acc_value = acc_corrected_result.truth_df[amp].values[0]
    thrown_value = thrown_result.truth_df[amp].values[0]    
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    ratios.append(ratio)

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(amplitudes, ratios)
ax.set_ylabel(r"$A_{\text{acc-corrected}} ~/~ A_{\text{thrown}}$", fontsize=18)
ax.set_title("Truth Fits: Ratio of Acceptance Corrected to Thrown Amplitudes")
ax.set_xticks(range(len(amplitudes)))
ax.set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes], rotation=45)
ax.set_ylim(0.9, 1.01)
plt.show()

There seems to be a common factor missing. We can notice that the number of events that we get when acceptance correcting does not match the number of events that we generated with

In [ ]:
print(f"Truth Thrown Generated Events: \t\t\t{thrown_result.fit_df['generated_events'].values[0]}")
print(f"Truth Acceptance Corrected Generated Events: \t{acc_corrected_result.fit_df['generated_events'].values[0]}")

percent_diff = 100 * (acc_corrected_result.fit_df['generated_events'].values[0] - thrown_result.fit_df['generated_events'].values[0]) / thrown_result.fit_df['generated_events'].values[0]
print(f"Percent Difference in Generated Events: \t{percent_diff:.2f}%")

If we divide out this value for each case, we can check if the ratios now deviate from 1

In [ ]:
amplitudes = utils.get_coherent_sums(acc_corrected_result.truth_df)["eJPmL"]

ratios = []
for amp in amplitudes:
    acc_value = acc_corrected_result.truth_df[amp].values[0] / acc_corrected_result.truth_df["generated_events"].values[0]
    thrown_value = thrown_result.truth_df[amp].values[0] / thrown_result.truth_df["generated_events"].values[0]
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    ratios.append(ratio)

if not np.allclose(ratios, 1.0, atol=1e-2):
    raise ValueError("Warning: Acceptance correction may not be working well, ratios deviate from 1 by more than 1%")    

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(amplitudes, ratios)
# \text{Intensity_{\text{acc-corrected}}}
ax.set_ylabel(r"$\dfrac{\left(\frac{A}{Intensity}\right)_{\text{acc-corrected}}}{\left(\frac{A}{Intensity}\right)_{\text{thrown}}} $", fontsize=18)
ax.set_title("Truth Fits: Ratio of Acceptance Corrected to Thrown Amplitude Fit Fractions")
ax.set_xticks(range(len(amplitudes)))
ax.set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes], rotation=45)
ax.set_ylim(0.8, 1.01)
plt.show()

Lastly, we can check the phase differences

In [ ]:
phase_diffs = list(utils.get_phase_differences(thrown_result.truth_df))

ratios = []
for phase_diff in phase_diffs:
    acc_value = acc_corrected_result.truth_df[phase_diff].values[0]
    thrown_value = thrown_result.truth_df[phase_diff].values[0]    
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan    
    ratios.append(ratio)

fig, ax = plt.subplots(figsize=(25, 5))
ax.bar(phase_diffs, ratios)
ax.set_ylabel(r"$\eta_{\text{acc-corrected}} ~/~ \eta_{\text{thrown}}$", fontsize=18)
ax.set_title("Truth Fits: Ratio of Acceptance Corrected to Thrown Phase Differences")
ax.set_xticks(range(len(phase_diffs)))
ax.set_xticklabels([utils.convert_amp_name(phase_diff) for phase_diff in phase_diffs], rotation=45, ha='right', rotation_mode='anchor')
ax.set_ylim(0.85, 1.01)
plt.show()

There appear to be two problematic phase differences, but we can see below that their values are extremely small. Since these are truth fits, they include Breit-Wigners which modulate the phase difference values. We corrected for this in the preprocess method of our ResultManager class earlier, but for small phase differences it likely introduces some error thats unaccounted for

In [ ]:
small_pds = ["p1ppD_p1m0P", "m1pmS_m1m0P"]
for phase_diff in small_pds:
    acc_value = acc_corrected_result.truth_df[phase_diff].values[0]
    thrown_value = thrown_result.truth_df[phase_diff].values[0]    
    print(f"{phase_diff}: acc={acc_value:.6f}, thrown={thrown_value:.6f}, ratio={acc_value/thrown_value:.6f}")

#### Projected Moments
Since the amplitudes are fixed, we should expect to find the same behavior in the projected moments

In [ ]:
moment_columns = [col for col in thrown_result.truth_proj_moments_df.columns if col.startswith("H")]

truth_moment_ratios = []
for m in moment_columns:
    acc_value = acc_corrected_result.truth_proj_moments_df[m].values[0]
    thrown_value = thrown_result.truth_proj_moments_df[m].values[0]
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    truth_moment_ratios.append(ratio)

fig, ax = plt.subplots(figsize=(20, 5))
ax.bar(moment_columns, truth_moment_ratios)
ax.set_ylabel(r"$H_{\text{acc-corrected}} ~/~ H_{\text{thrown}}$", fontsize=18)
ax.set_title("Ratio of Acceptance Corrected to Thrown Projected Moments")
ax.set_xticks(range(len(moment_columns)))
ax.set_xticklabels(moment_columns, rotation=90)
ax.set_ylim(0.9, 1.01)
plt.show()

Now perform the same division by the relative number of events

In [ ]:
truth_moment_ratios = []
for m in moment_columns:
    acc_value = acc_corrected_result.truth_proj_moments_df[m].values[0] / acc_corrected_result.truth_df["generated_events"].values[0]
    thrown_value = thrown_result.truth_proj_moments_df[m].values[0] / thrown_result.truth_df["generated_events"].values[0]
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    truth_moment_ratios.append(ratio)

if not np.allclose(truth_moment_ratios, 1.0, atol=1e-2):
    raise ValueError("Warning: Acceptance correction may not be working well, ratios deviate from 1 by more than 1%")

fig, ax = plt.subplots(figsize=(20, 5))
ax.bar(moment_columns, truth_moment_ratios)
ax.set_ylabel(r"$\dfrac{\left(\frac{H}{Intensity}\right)_{\text{acc-corrected}}}{\left(\frac{H}{Intensity}\right)_{\text{thrown}}} $", fontsize=18)
ax.set_title("Ratio of Acceptance Corrected to Thrown Projected Moment Fit Fractions")
ax.set_xticks(range(len(moment_columns)))
ax.set_xticklabels(moment_columns, rotation=90)
ax.set_ylim(0.9, 1.01)
plt.show()

### Evaluating the Fit Result to the Accepted Monte Carlo
With acceptable acceptance-correction from the previous section, we can turn our attention to the fit results in the accepted Monte Carlo. Here we have performed 100 randomized fit results, in which we will only look at the fit result with the best likelihood out of those 100. This "best fit" was used to seed 300 bootstrap fits so that we can obtain more reasonable errors on the partial wave results. The projected moments currently have no method for error propagation from the PWA results, so the bootstrap also provides us with projected moment errors we would otherwise lack.

#### Accepted Partial Wave Result
To understand how the result of our partial wave fit performed, we can compare it to the truth information. We'll do this first for the acceptance-included Monte Carlo, just to avoid any possible issues in the acceptance correction. We'll do the comparison by calculating the weighted residuals
$$
\frac{A_{\text{acc}} - A_{\text{truth-acc}}}{\sigma_{A,\text{acc}}}
\,,
$$
but to give the values more context, we'll present them alongside the fit fractions and relative uncertainties. If uncertainties are on the same order of their value ($\sigma_x/x\approx 1$), then the weighted residual will be quite small, but not as accurate a representation of model performance. We expect high relative uncertainties for extremely small amplitudes though, i.e. amplitudes with a very small fit fraction.

In [ ]:
amplitude = utils.get_coherent_sums(acc_result.fit_df)["eJPmL"]

fit_fractions = []
relative_uncertainties = []
weighted_residuals = []
intensity = acc_result.fit_df.loc[0, "detected_events"]

for amp in amplitudes:
    value = acc_result.fit_df.loc[0, amp]
    err = acc_result.bootstrap_df[f"{amp}"].std()
    fit_frac = value / intensity if intensity != 0 else np.nan
    fit_fractions.append(fit_frac)

    rel_unc = abs(err / value) if value != 0 else np.nan
    relative_uncertainties.append(rel_unc)
    
    true_val = acc_result.truth_df.loc[0, amp]
    w_res = (value - true_val) / err if err != 0 else np.nan
    weighted_residuals.append(w_res)

fig, axs = plt.subplots(3, 1, figsize=(15, 6), sharex=True)

# fit fractions
axs[0].bar(amplitudes, fit_fractions, yerr=[
    acc_result.bootstrap_df[amp].std() / intensity if intensity != 0 else 0 for amp in amplitudes
])
axs[0].set_xticks(range(len(amplitudes)))
axs[0].set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes])
axs[0].set_ylabel(r"$|A|^2 /$ Intensity", fontsize=14)
axs[0].tick_params(axis='x', rotation=45)
axs[0].set_title("Fit Fractions")

# relative uncertainties
axs[1].bar(amplitudes, relative_uncertainties)
axs[1].set_ylabel(r"$\sigma_{|A|^2} ~/~|A|^2$", fontsize=16)
axs[1].tick_params(axis='x', rotation=45)
axs[1].set_title("Relative Uncertainty")

# weighted residuals
axs[2].bar(amplitudes, weighted_residuals)
axs[2].set_ylabel(r"$\frac{A_{\text{acc}} - A_{\text{true}}}{\sigma_{A_{\text{acc}}}}$", fontsize=20)
axs[2].set_xticks(range(len(amplitudes)))
axs[2].set_xticklabels([utils.convert_amp_name(label) for label in amplitudes], rotation=45, ha='right', rotation_mode='anchor')
axs[2].axhline(0, color='k', linestyle='-', linewidth=1)
axs[2].axhline(3, color='r', linestyle='--', linewidth=1)
axs[2].axhline(-3, color='r', linestyle='--', linewidth=1)
axs[2].set_ylim(-max(abs(np.array(weighted_residuals))) - 0.1, max(abs(np.array(weighted_residuals))) + 0.1)
axs[2].set_title(rf"Weighted Residuals")

plt.tight_layout()
plt.show()

In [ ]:
phase_diffs = list(utils.get_phase_differences(acc_result.fit_df))
avg_fit_fractions = []
relative_uncertainties_pd = []
weighted_residuals = []

intensity = acc_result.fit_df.loc[0, "detected_events"]
for phase_diff in phase_diffs:
    # Get the two amplitudes involved in the phase difference to get average fit fraction
    amp1, amp2 = phase_diff.split('_')
    fit_frac1 = acc_result.fit_df.loc[0, amp1] / intensity if intensity != 0 else np.nan
    fit_frac2 = acc_result.fit_df.loc[0, amp2] / intensity if intensity != 0 else np.nan
    avg_fit_fractions.append(0.5 * (abs(fit_frac1) + abs(fit_frac2)))

    val = acc_result.fit_df.loc[0, phase_diff]    
    # Use circular standard deviation for phase differences, but need to convert to radians, then back to degrees
    val_err = np.rad2deg(
        scipy.stats.circstd(
            np.deg2rad(acc_result.bootstrap_df[phase_diff]), low=-np.pi, high=np.pi
        )
    )
    rel_unc = abs(val_err / val) if val != 0 else np.nan
    relative_uncertainties_pd.append(rel_unc)

    true_val = acc_result.truth_df.loc[0, phase_diff]
    w_res = utils.circular_residual(val, true_val, True) / val_err if val_err != 0 else np.nan
    weighted_residuals.append(w_res)

fig, axs = plt.subplots(3, 1, figsize=(18, 8), sharex=True)

axs[0].bar(phase_diffs, avg_fit_fractions)
axs[0].set_ylabel(r"$\frac{|A_1|^2 + |A_2|^2}{2 \times \text{Intensity}}$", fontsize=20)
axs[0].set_title("Average Fit Fraction of Amplitudes in Each Phase Difference")
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

axs[1].bar(phase_diffs, relative_uncertainties_pd)
axs[1].set_ylabel(r"$\sigma_{\eta_{A_1, A_2}} / \eta_{A_1, A_2}$", fontsize=20)
axs[1].set_title("Relative Uncertainty")
axs[1].tick_params(axis='x', rotation=45)
axs[1].set_yscale('log')

axs[2].bar(phase_diffs, weighted_residuals)
axs[2].set_ylabel(r"$\frac{\eta_{\text{acc}} - \eta_{\text{true}}}{\sigma_{\eta_{\text{acc}}}}$", fontsize=20)
axs[2].axhline(0, color='k', linestyle='-', linewidth=1)
axs[2].axhline(3, color='r', linestyle='--', linewidth=1)
axs[2].axhline(-3, color='r', linestyle='--', linewidth=1)
axs[2].set_ylim(-max(abs(np.array(weighted_residuals))) - 0.1, max(abs(np.array(weighted_residuals))) + 0.1)
axs[2].set_title(rf"Weighted Residuals")
axs[2].set_xticks(range(len(phase_diffs)))
axs[2].set_xticklabels([utils.convert_amp_name(phase_diff) for phase_diff in phase_diffs], rotation=45, ha="right", rotation_mode="anchor")

plt.tight_layout()
plt.show()

#### Acceptance-Corrected Partial Wave Result
Now we can repeat the same procedure, but use the acceptance-corrected version of the values and compare them to the truth thrown values. Remember that we are simply accepting-correcting the values in the previous section, so these are not necessarily different fits being compared. Any issues found in the non-acceptance corrected values will likely persist here. The main component being changed here is that the true values we are comparing to are those from the thrown dataset

In [ ]:
amplitude = utils.get_coherent_sums(acc_corrected_result.fit_df)["eJPmL"]

fit_fractions = []
fit_fractions_err = []
relative_uncertainties = []
weighted_residuals = []
intensity = acc_corrected_result.fit_df.loc[0, "generated_events"]
true_intensity = acc_corrected_result.truth_df.loc[0, "generated_events"]

for amp in amplitudes:
    value = acc_corrected_result.fit_df.loc[0, amp]
    err = acc_corrected_result.bootstrap_df[f"{amp}"].std()
    fit_frac = value / intensity if intensity != 0 else np.nan
    fit_frac_err = err / intensity if intensity != 0 else np.nan
    fit_fractions.append(fit_frac)
    fit_fractions_err.append(fit_frac_err)

    rel_unc = abs(err / value) if value != 0 else np.nan
    relative_uncertainties.append(rel_unc)

    true_val = thrown_result.truth_df.loc[0, amp]
    w_res = (fit_frac - (true_val / true_intensity)) / fit_frac_err if fit_frac_err != 0 else np.nan
    weighted_residuals.append(w_res)

fig, axs = plt.subplots(3, 1, figsize=(15, 6), sharex=True)

# fit fractions
axs[0].bar(amplitudes, fit_fractions, yerr=fit_fractions_err)
axs[0].set_ylabel(r"$|A|^2 /$ Intensity", fontsize=14)
axs[0].set_title("Fit Fractions")

# relative uncertainties
axs[1].bar(amplitudes, relative_uncertainties)
axs[1].set_ylabel(r"$\sigma_{|A|^2} ~/~|A|^2$", fontsize=16)
axs[1].set_title("Relative Uncertainty")

# weighted residuals
axs[2].bar(amplitudes, weighted_residuals)
axs[2].set_ylabel(r"$\frac{A'_{\text{acc}} - A'_{\text{true}}}{\sigma_{A'_{\text{acc}}}}$", fontsize=20)
axs[2].set_xticks(range(len(amplitudes)))
axs[2].set_xticklabels([utils.convert_amp_name(label) for label in amplitudes], rotation=45, ha='right', rotation_mode='anchor')
axs[2].axhline(0, color='k', linestyle='-', linewidth=1)
axs[2].axhline(3, color='r', linestyle='--', linewidth=1)
axs[2].axhline(-3, color='r', linestyle='--', linewidth=1)
axs[2].set_ylim(-max(abs(np.array(weighted_residuals))) - 0.1, max(abs(np.array(weighted_residuals))) + 0.1)
axs[2].set_title(rf"Weighted Residuals")

plt.tight_layout()
plt.show()

In [ ]:
acc_corrected_result.plot.bootstrap().pairplot([0], amplitudes, is_acceptance_corrected=True)

In [ ]:
phase_diffs = list(utils.get_phase_differences(acc_corrected_result.fit_df))
avg_fit_fractions = []
relative_uncertainties_pd = []
weighted_residuals = []

intensity = acc_corrected_result.fit_df.loc[0, "generated_events"]
for phase_diff in phase_diffs:
    # Get the two amplitudes involved in the phase difference to get average fit fraction
    amp1, amp2 = phase_diff.split('_')
    fit_frac1 = acc_corrected_result.fit_df.loc[0, amp1] / intensity if intensity != 0 else np.nan
    fit_frac2 = acc_corrected_result.fit_df.loc[0, amp2] / intensity if intensity != 0 else np.nan
    avg_fit_fractions.append(0.5 * (abs(fit_frac1) + abs(fit_frac2)))

    val = acc_corrected_result.fit_df.loc[0, phase_diff]
    # Use circular standard deviation for phase differences, but need to convert to radians, then back to degrees
    val_err = np.rad2deg(
        scipy.stats.circstd(
            np.deg2rad(acc_corrected_result.bootstrap_df[phase_diff]), low=-np.pi, high=np.pi
        )
    )
    rel_unc = abs(val_err / val) if val != 0 else np.nan
    relative_uncertainties_pd.append(rel_unc)

    true_val = thrown_result.truth_df.loc[0, phase_diff]
    w_res = utils.circular_residual(val, true_val, True) / val_err if val_err != 0 else np.nan
    weighted_residuals.append(w_res)

fig, axs = plt.subplots(3, 1, figsize=(18, 8), sharex=True)

axs[0].bar(phase_diffs, avg_fit_fractions)
axs[0].set_ylabel(r"$\frac{|A_1|^2 + |A_2|^2}{2 \times \text{Intensity}}$", fontsize=20)
axs[0].set_title("Average Fit Fraction of Amplitudes in Each Phase Difference")
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

axs[1].bar(phase_diffs, relative_uncertainties_pd)
axs[1].set_ylabel(r"$\sigma_{\eta_{A_1, A_2}} / \eta_{A_1, A_2}$", fontsize=20)
axs[1].set_title("Relative Uncertainty")
axs[1].tick_params(axis='x', rotation=45)
axs[1].set_yscale('log')

axs[2].bar(phase_diffs, weighted_residuals)
axs[2].set_ylabel(r"$\frac{\eta_{\text{acc}} - \eta_{\text{true}}}{\sigma_{\eta_{\text{acc}}}}$", fontsize=20)
axs[2].axhline(0, color='k', linestyle='-', linewidth=1)
axs[2].axhline(3, color='r', linestyle='--', linewidth=1)
axs[2].axhline(-3, color='r', linestyle='--', linewidth=1)
axs[2].set_ylim(-max(abs(np.array(weighted_residuals))) - 0.1, max(abs(np.array(weighted_residuals))) + 0.1)
axs[2].set_title(rf"Weighted Residuals")
axs[2].set_xticks(range(len(phase_diffs)))
axs[2].set_xticklabels([utils.convert_amp_name(phase_diff) for phase_diff in phase_diffs], rotation=45, ha="right", rotation_mode="anchor")

plt.tight_layout()
plt.show()

In [ ]:
# acc_corrected_result.plot.bootstrap().pairplot([0], phase_diffs)

#### Acceptance Corrected Projected Moments
We can now compare how the acceptance-corrected projected moments compare to the true thrown projected moments. If our partial wave model did not resolve to the true values, but the projected moments do, then it may hint at an ambiguous solution. If our moments do not match though, then it indicates we have simply not obtained the best solution. We'll prescribe the same method, checking the normalized moments $H' = H/H^0(0,0,0,0)$ (similar to a fit fraction due to $H^0(0,0,0,0) =$ # of events), relative uncertainty, and weighted residual with respect to the true thrown projected moments.

Due to the mismatch in the number of events between thrown and acceptance-corrected data, we'll need to use the normalized moments in the weighted residual as well.

In [ ]:
moments = [col for col in thrown_result.truth_proj_moments_df.columns if col.startswith("H")]

fit_fractions = []
fit_fractions_err = []
relative_uncertainties = []
weighted_residuals = []

H0_0000 = acc_corrected_result.proj_moments_df.loc[0, "H0_0000"]
true_H0_0000 = thrown_result.truth_proj_moments_df.loc[0, "H0_0000"]
for mom in moments:
    value = acc_corrected_result.proj_moments_df.loc[0, mom]
    err = acc_corrected_result.bootstrap_proj_moments_df[mom].std()

    fit_frac = value / H0_0000 if H0_0000 != 0 else np.nan
    fit_frac_err = err / H0_0000 if H0_0000 != 0 else np.nan
    fit_fractions.append(fit_frac)
    fit_fractions_err.append(fit_frac_err)

    rel_unc = abs(err / value) if value != 0 else np.nan    
    relative_uncertainties.append(rel_unc)

    true_val = thrown_result.truth_proj_moments_df.loc[0, mom]
    w_res = (fit_frac - (true_val / true_H0_0000)) / fit_frac_err if fit_frac_err != 0 else np.nan
    weighted_residuals.append(w_res)

fig, axs = plt.subplots(3, 1, figsize=(15, 6), sharex=True)

# normalized moments
axs[0].bar(moments, np.abs(np.array(fit_fractions)), yerr=fit_fractions_err) # use absolute value so we can plot on log scale
axs[0].set_ylabel(r"$|H ~/~ H0\_0000|$ ")
axs[0].set_title("Normalized Moments")
axs[0].set_yscale('log') # plot log since some moments are very tiny

# relative uncertainties
axs[1].bar(moments, relative_uncertainties)
axs[1].set_ylabel(r"$\sigma_{H} ~/~ H$")
axs[1].set_title("Relative Uncertainty")
axs[1].set_yscale('log')

axs[2].bar(moments, weighted_residuals)
axs[2].set_ylabel(r"$\frac{H'_{\text{acc}} - H'_{\text{true}}}{\sigma_{H'_{\text{acc}}}}$", fontsize=20)
axs[2].axhline(0, color='k', linestyle='-', linewidth=1)
axs[2].axhline(3, color='r', linestyle='--', linewidth=1)
axs[2].axhline(-3, color='r', linestyle='--', linewidth=1)
axs[2].set_ylim(-max(abs(np.array(weighted_residuals))) - 0.1, max(abs(np.array(weighted_residuals))) + 0.1)
axs[2].set_title(rf"Weighted Residuals")
axs[2].set_xticks(range(len(moments)))
axs[2].set_xticklabels([utils.convert_moment_name(mom) for mom in moments], rotation=45, ha='right', rotation_mode='anchor')

plt.tight_layout()
plt.show()

Some of these moments are clearly very small, and so it may not be realistic to expect that we project their values well. Lets look at just the moments whose normalized values are "signficant" $|H/H^0(0000)| > 0.01$

In [ ]:
moments = [col for col in thrown_result.truth_proj_moments_df.columns if col.startswith("H")]

true_fit_fractions = [thrown_result.truth_proj_moments_df.loc[0, mom] / thrown_result.truth_proj_moments_df.loc[0, "H0_0000"] for mom in moments]

mask = np.abs(np.array(true_fit_fractions)) > 0.01

fig, axs = plt.subplots(3, 1, figsize=(15, 6), sharex=True)

# normalized moments
axs[0].bar(np.array(moments)[mask], np.abs(np.array(fit_fractions)[mask]), yerr=np.array(fit_fractions_err)[mask]) # use absolute value so we can plot on log scale
axs[0].set_ylabel(r"$|H ~/~ H0\_0000|$ ")
axs[0].set_title("Normalized Moments")
axs[0].set_yscale('log') # plot log since some moments are very tiny

# relative uncertainties
axs[1].bar(np.array(moments)[mask], np.array(relative_uncertainties)[mask])
axs[1].set_ylabel(r"$\sigma_{H} ~/~ H$")
axs[1].set_title("Relative Uncertainty")
axs[1].set_yscale('log')

axs[2].bar(np.array(moments)[mask], np.array(weighted_residuals)[mask])
axs[2].set_ylabel(r"$\frac{H'_{\text{acc}} - H'_{\text{true}}}{\sigma_{H'_{\text{acc}}}}$", fontsize=20)
axs[2].axhline(0, color='k', linestyle='-', linewidth=1)
axs[2].axhline(3, color='r', linestyle='--', linewidth=1)
axs[2].axhline(-3, color='r', linestyle='--', linewidth=1)
axs[2].set_ylim(-max(abs(np.array(weighted_residuals)[mask])) - 0.1, max(abs(np.array(weighted_residuals)[mask])) + 0.1)
axs[2].set_title(rf"Weighted Residuals")
axs[2].set_xticks(range(len(np.array(moments)[mask])))
axs[2].set_xticklabels([utils.convert_moment_name(mom) for mom in np.array(moments)[mask]], rotation=45, ha='right', rotation_mode='anchor')


plt.tight_layout()
plt.show()

### Evaluating Randomized Fits with Moments
As an added bonus, we can use the moments to see whether our best fit is truly a unique result. We can do this by comparing the $\Delta\ln\mathscr{L}$ vs $\chi^2_{H} / ndf$.

In [ ]:
moments = [col for col in acc_corrected_result.randomized_proj_moments_df.columns if col.startswith("H")]
# using acc_corrected result since moments are always acceptance corrected, so they're comparable to rand projected moments
best_moments = acc_corrected_result.proj_moments_df.loc[0, moments].values
best_moments_err = acc_corrected_result.bootstrap_proj_moments_df[moments].std().values
best_likelihood = acc_corrected_result.fit_df.loc[0, "likelihood"]

In [ ]:
moment_averaged_residuals = []
rand_likelihoods = []

for i in range(len(acc_corrected_result.randomized_proj_moments_df)):
    rand_moments = acc_corrected_result.randomized_proj_moments_df.loc[i, moments].values

    squared_weighted_residual = (
        np.sum(
            ((best_moments - rand_moments) ** 2) / (best_moments_err)**2)
    ) / len(moments)
    moment_averaged_residuals.append(squared_weighted_residual)

    rand_likelihood = acc_corrected_result.randomized_df.loc[i, "likelihood"]
    rand_likelihoods.append(rand_likelihood)

In [ ]:
# Calculate the difference in likelihood between the best and each randomized fit
delta_lnL = [ll - best_likelihood for ll in rand_likelihoods]

plt.figure(figsize=(8, 6))
plt.scatter(delta_lnL, moment_averaged_residuals, c='tab:blue', s=20, alpha=0.7)
plt.xlabel(r'$\Delta \ln \mathcal{L}$ (randomized - best)', fontsize=14)
plt.ylabel(r'$\frac{1}{n}\sum^n \frac{(H_n^{\text{best}} - H_n^{\text{rand}})^2}{(\sigma_{H_n}^{\text{best}})^2}$', fontsize=25)
plt.title(r'$\Delta \ln \mathcal{L}$ vs. $\chi^2_{H}$ for Randomized Fits', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Find indices in randomized_df where likelihood is approximately -41645.6 (2nd best value)
matching_indices = acc_corrected_result.randomized_df.index[
    np.isclose(acc_corrected_result.randomized_df["likelihood"], -41645.6)
]

# Select corresponding rows from randomized_proj_moments_df
matching_proj_moments = acc_corrected_result.randomized_proj_moments_df.loc[matching_indices]

In [ ]:
# Plot weighted residuals for each matching_proj_moments using the best fit as the comparison
for i in range(len(matching_proj_moments)):
    rand_moments = matching_proj_moments.iloc[i][moments].values
    # Avoid division by zero in error    
    weighted_residual = (best_moments - rand_moments) / best_moments_err

    plt.figure(figsize=(20, 5))
    plt.bar(moments, weighted_residual)
    plt.axhline(0, color='k', linestyle='-', linewidth=1)
    plt.axhline(3, color='r', linestyle='--', linewidth=1)
    plt.axhline(-3, color='r', linestyle='--', linewidth=1)
    plt.xticks(rotation=90)
    plt.ylabel(r'Weighted Residual: $(H^{\text{best}} - H^{\text{rand}})/\sigma_{H^{\text{best}}}$', fontsize=16)
    plt.title(f'Weighted Residuals: Best Fit vs. Randomized Fit {matching_indices[i]}', fontsize=18)
    plt.tight_layout()
    plt.show()